# Latar Belakang
TSA atau *Transportation Security Administration* adalah sebuah badan pemerintah dari departemen keamanan dalam negeri Amerika Serikat (USA) yang bertanggung jawab dalam masalah keamanan penerbangan. Klaim dapat diajukan jika penumpang terluka atau harta bendanya hilang atau rusak selama proses pemeriksaan di bandara. Banyak klaim yang masuk, tapi tidak semua klaim akan diproses dan disetujui.

## Pernyataan Masalah
TSA ingin mengetahui keputusan/penyelesaian akhir (*final settlement/Disposition*) apa yang harus diberikan dari sebuah klaim yang diajukan. Dimana ada tiga kategori untuk keputusan/penyelesaian akhir :

1. ***Approve in Full (Disetujui secara penuh)***   : klaim disetujui dan dana kompensasi diberikan secara penuh (full) sesuai dengan pengajuan klaim penumpang.
2. ***Settle (Disetujui secara parsial/sebagian)*** : klaim telah terselesaikan dengan persetujuan antara pihak agensi TSA dan penumpang. Dana kompensasi diberikan secara sebagian (partial)
3. ***Deny (Ditolak)***                             : klaim ditolak atau tidak disetujui.


Informasi ini diharapkan dapat membantu TSA untuk menekan biaya kompensasi dan waktu pengecekan klaim, mendeteksi adanya klaim fiktif atau palsu, memberikan evaluasi kepada bandara dalam hal penanganan klaim dan meningkatkan kualitas dengan membantu mengelompokkan klaim ke dalam keputusan akhir (*final settlement*) yang sesuai.

Sebagai seorang *data analyst*, saya akan mencoba menjawab pertanyaan berikut:

**Bagaimana karakteristik keputusan/penyelesaian akhir yang diambil terhadap klaim yang diajukan?**

# Data
Untuk menjawab pertanyaan di atas, saya akan menganalisa data klaim yang sudah dikumpulkan oleh TSA. Dataset dapat diakses [di sini](https://drive.google.com/drive/folders/13SAQcA3QZ2FBclO1iOW31otlnBXHytYk).

In [ ]:
import pandas as pd
import numpy as np
import re
import datetime

import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt

# import warnings
# warnings.filterwarnings("ignore")

Dataset ini berisi informasi terkait jenis klaim, waktu kejadian dan pelaporan klaim, lokasi bandara, maskapai, status dan biaya klaim (dolar US). \
Ada 13 kolom di dalam data tsa_claims, yaitu

* Claim Number  : nomor id unik untuk setiap klaim
* Date Received : tanggal klaim diajukan
* Incident Date : tanggal kejadian tersebut terjadi
* Airport Code  : kode bandara dimana kejadian tersebut terjadi
* Airport Name  : nama bandara dimana kejadian tersebut terjadi
* Airline Name  : nama maskapai/penyedia penerbangan
* Claim Type    : tipe klaim yang diajukan
* Claim Site    : lokasi dimana klaim diajukan
* Item          : jenis benda yang di klaim
* Claim Amount  : jumlah dana (dolar US) klaim 
* Status        : status klaim yang diajukan (apakah sudah selesai, ditangguhkan , ditunda dll)
* Close Amount  : jumlah dana kompensasi penyelesaian akhir (settlement)
* Disposition   : status penyelesaian akhir ( *final settlement*) klaim

Berikut 5 baris teratas dan terbawah dari dataset tsa_claims.

In [ ]:
df = pd.read_csv('tsa_claims.csv')
display(df.head(),df.tail())

## Data Understanding and Cleaning
Sebelum melakukan analisis lebih mendalam, kita perlu mengenal data yang kita miliki sekarang melalui tahapan *Data Understanding*. Dari proses ini kita bisa mengetahui anomali-anomali yang terdapat pada dataset seperti *missing value*, format data yang tidak sesuai, data duplikat, penamaan variabel yang tidak konsisten dll. Anomali-anomali yang berhasil dideteksi perlu ditangani dalam tahapan *data cleaning*, dimana pada setiap pengangan anomali tersebut akan dilakukan penyesuaian langkah yang diambil (tergantung dari jenis anomalinya) baik secara *domain knowledge* maupun secara *statistik* sehingga data tersebut menjadi "bersih" dan lebih relevan untuk digunakan dalam proses analisis.

### Deskripsi data

Pertama, mari kita lihat informasi umur dari dataset tsa_claims.

In [ ]:
print(df.shape)
df.info()

Dari info di atas dapat kita lihat bahwa semua data masih memiliki tipe *object* atau teks, sehingga kita perlu menyesuaikan format data yang relevan.

* Beberapa kolom yaitu, `Claim Number`, `Airport Code`, `Airport Name`, `Airline Name`, `Claim Type`, `Claim Site`, `Item`, `Status`, dan `Disposition` tetap dalam tipe teks/object.
* kolom `Date Received`, dan `Incident Date` akan diubah ke tipe *datetime*.
* kolom `Claim Amount` dan `Close Amount` akan diubah ke tipe *float*.

In [ ]:
print("Deskripsi statistik data numerik")
display(df.describe())
print("*"*200)
print("Deskripsi statistik data non-numerik (object)")
display(df.describe(include='object'))

In [ ]:
pd.set_option('display.max_colwidth', 1)

df.shape
# data unik di tiap kolom
list_unique_item = [[col, df[col].nunique(), df[col].unique()] for col in df.columns]

tabel_unique_desc = pd.DataFrame(columns=['Column Name', 'Number of Unique', 'Unique Sample'],
                     data=list_unique_item)
tabel_unique_desc

In [ ]:
#kolom yang terdapat karakter "-" yang dapat diartikan sebagai nilai kosong
list_of_col_with_strip = []
for col in df.columns :
    if "-" in df[col].values :
        list_of_col_with_strip.append(col)
list_of_col_with_strip

Jika dilihat, ada beberapa kolom (`Airport Code`,`Airport Name`, `Airline Name`, `Claim Type`, `Claim Site`, `Item`,`Claim Amount`,`Status`) yang memiliki nilai kosong (missing value) menggunakan karakter "-". \
Maka nilai "-" akan di replace dengan nilai Nan

In [ ]:
for col in df.columns:
    df.loc[df[col] == "-", col] = df.loc[df[col] == "-", col].replace("-", np.nan)

Dari hasil penjabaran di atas, maka secara umum kita dapat melihat bahwa :

* Dataset tsa_claims memiliki 13 kolom dan 204.267 baris
* Kolom `Claim Number` memiliki 204.258 data unik, yang berarti bahwa ada 9 baris data dengan nomor id yang sama. Mari nanti kita cek lebih detil. \
  Nantinya, setelah dilakukan penyesuaian data duplikat, karena kolom ini berisikan id dari klaim, karena tidak relevan dalam analisis maka di drop saja. 
* Semua kolom kecuali `Claim Number` memiliki nilai kosong yang diwakilkan dengan data NaN (setelah karakter "-" di replace dengan data NaN).
* Kolom `Date Received` dan `Incident Date` memiliki data tipe object. Seharusnya berisikan data tipe datetime. Kita akan cek lebih detil.
* Terdapat penulisan data yang tidak konsisten pada kolom `Incident Date`. contoh : 22-Apr-13 atau 4/22/2013 8:40. Kita akan cek lebih detil
* Kolom `Claim Amount` dan `Close Amount` memiliki data tipe object. Seharusnya berisikan data numerik. Kita akan cek lebih detil.
* Untuk nilai di atas $1000 pada kolom `Claim Amount` dan `Close Amount` : terdapat karakter ";" yang dimana nanti akan kita sesuaikan formatnya.
* Pada kolom `Item` terdapat banyak teks/kata yang menunjukkan nama-nama barang yang dijadikan satu *field*. Kita bisa estrak satu persatu. contoh :  Cosmetics - Perfume; \ toilet articles; medicines; soaps; etc.; Locks; Medicines
* Ada beberapa nilai pada kolom `status` yang memiliki arti yang sama jika dihubungkan dengan nilai di kolom `Disposition`. \
  Contoh : "Denied" dan "Deny" memiliki arti yang sama dengan "Deny" pada kolom `Disposition`. Kita bisa gunakan kondisi ini untuk mengisi nilai kosong pada `Disposition`


### Data Duplikat

In [ ]:
df[df['Claim Number'].duplicated(keep=False)].sort_values(by='Claim Number')

Jika kita lihat ada 9 data yang memiliki nomor id klaim atau Claim Number yang sama. Ada dua kemungkinan, pertama yaitu ketika data yang di masukkan memang data yang sama/duplikat \
atau ada klaim yang sama (dari orang yang sama) diajukan dua kali di waktu yang berbeda. Pada kasus kedua, kita akan drop baris yang memiliki nilai kosong pada kolom `Status`, dengan asumsi klaim yang sama tersebut tidak diproses oleh TSA

In [ ]:
idx_to_drop = df[df['Claim Number'].duplicated(keep=False)].loc[df['Status'].isnull()].index
df = df.drop(idx_to_drop, axis=0)
df = df.drop_duplicates()
df.reset_index(inplace=True, drop='index')

In [ ]:
#Sudah tidak ada data dengan nomor id atau Claim Number duplikat 
df[df['Claim Number'].duplicated()]

### Justifikasi Format Data

Seperti yang kita sudah lihat, bahwa ada beberapa kolom yang harus kita sesuaikan baik tipe data nya maupun penulisan/penamaan nilai.

* beberapa data pada kolom `Incident Date` mempunyai penulisan yang tidak konsisten, sehingga kita akan sesuaikan agar seragam. Contoh : "4/22/2013 8:40" akan diubah menjadi "2013-04-22" (dalam format %Y-%m-%d)
* kolom `Date Received` dan `Incident Date` memiliki data tipe object yang akan diubah ke tipe datetime.
* kolom `Claim Amount` dan `Close Amount` memiliki data tipe object yang seharusnya berisikan data numerik --> tipe "int" atau tipe "float", disini kita akan ubah tipe data nya menjadi tipe "float"


In [ ]:
a= df.copy()

#### Ubah penulisan dan tipe data pada kolom `Date Received` dan `Incident Date`

1) kolom `Date Received`

In [ ]:
df['Date Received'] = pd.to_datetime(df['Date Received']).dt.strftime('%Y-%m-%d')

2) kolom `Incident Date`

Saat hendak mengubah kolom "Incident Date" muncul pesan error \
```OutOfBoundsDatetime: Out of bounds nanosecond timestamp: 300-09-03 00:00:00 present at position 741```

Ini disebabkan ada beberapa data melebihi range nilai yang diizinkan oleh library pandas \
```In [0]: pd.Timestamp.min```
```Out[0]: Timestamp('1677-09-22 00:12:43.145225')```

```In [55]: pd.Timestamp.max```
```Out[55]: Timestamp('2262-04-11 23:47:16.854775807')```

Kita akan cek baris yang melebihi range ini

In [ ]:
df[df['Incident Date']=="03-SEP-0300 00:00"]

dari sampel di atas dapat dilihat bahwa ada format dari kolom `Incident Date` yang format tahun nya tidak sesuai \
contoh di atas : *03-SEP-0300 00:00* \
Nilai tahun (0300) pada data tersebut tidak dimengerti --> mungkin ada kesalahan input atau perbedaan format. \
Untuk kasus ini, kita asumsikan antara tanggal pengajuan klaim (Date Received) dan tanggal kejadian (Incident Date) terjadi di tahun yang sama sehingga nanti kita akan replace tahun pada kolom `Incident Date`  menggunakan tahun di kolom `Date Received`

In [ ]:
row_wrong_format= list(df['Incident Date'].str.extract(pat=r"(\d+-\D+-\d{4})").dropna().index.to_numpy())
df.iloc[row_wrong_format][['Date Received', 'Incident Date']]

Dapat dilihat dari hasil di atas, ada 164 baris yang memiliki nilai range datetime (dalam ns) yang melebihi batas/range pandas \
kita akan ubah nilai tahun nya menggunakan nilai tahun pada kolom `Date Received`

In [ ]:
year_list_date_received = pd.to_datetime(df.loc[row_wrong_format, 'Date Received']).dt.year.values.tolist()

In [ ]:
for idx, year in zip(row_wrong_format, year_list_date_received):
    teks = df.loc[idx, 'Incident Date']
    a = teks.split("-")
    a[1] = a[1].capitalize()
    new_teks = "-".join(a[0:3])

    df.loc[[idx], 'Incident Date'] = new_teks
    df.loc[[idx], 'Incident Date'] = df.loc[[idx], 'Incident Date'].apply(lambda x : re.sub(("-(\d{4})"), f"-{year}", x))

Setelah format tahunnya disesuaikan pada kolom ``Incident Date``, selanjutnya ubah ke format tanggal yang sesuai.

In [ ]:
df['Incident Date'] = pd.to_datetime(df['Incident Date']).dt.date

In [ ]:
df.iloc[row_wrong_format][['Date Received', 'Incident Date']]

#### Ubah penulisan dan tipe data pada kolom `Claim Amount` dan `Close Amount`

In [ ]:
df[['Claim Amount', 'Close Amount']].sample(10)

replace tanda ";" untuk setiap nilai dengan ""

In [ ]:
df[['Claim Amount', 'Close Amount']] = df[['Claim Amount', 'Close Amount']].replace({";": ""}, regex=True)

Selanjutnya extrak angka numerik di depan tanda dollar $ lalu ubah tipe data nya ke data tipe *float*

In [ ]:
df['Claim Amount'] = df['Claim Amount'].str.replace("$", "", regex=True)
df['Close Amount'] = df['Close Amount'].str.replace("$", "", regex=True)

In [ ]:
df[['Claim Amount', 'Close Amount']] = df[['Claim Amount', 'Close Amount']].astype(float)
df[['Claim Amount', 'Close Amount']].head()

In [ ]:
df.sample(10)

### Missing Value Handling

In [ ]:
#persentase missing value setiap kolom
missing_pct_df = pd.DataFrame({"column name" : df.columns.tolist(), "percentage (missing value)" : [df[col].isnull().sum()*100/len(df) for col in df.columns]}).sort_values(by="percentage (missing value)", ascending=False)
missing_pct_df

In [ ]:
# Sebaran missing value di dalam data tsa_claims
px.imshow(df[df.columns.tolist()[1:]].isnull(), height=600, width=1500,color_continuous_scale='ice')

Dari penjabaran di atas, dapat kita lihat bahwa :

* terdapat 3 kolom yang memiliki proporsi *missing value* cukup tinggi (>10%) yaitu `Disposition` (35.69%), `Close Amount` (33.75%) dan `Airline Name` (18.90%)
* *missing value* pada kolom `Close Amount` dan `Disposition` memiliki sebaran yang terlihat berkaitan, jika data di `Close Amount` kosong, maka kemungkinan besar data di `Disposition` juga kosong.
* *missing value* pada kolom `Claim Amount` dan ``Close Amount`` terlihat cukup berkaitan dengan kolom `Status`, jika data di `Close Amount` dan `Claim Amount` keduanya kosong, maka kemungkinan data di `Status` juga kosong (terutama pada data di index 14000 ke atas)
* proporsi *missing value* pada kolom `Date Received` (0.128%) dan `Claim Site` (0.497%) terbilang kecil (<1%>) , ada kemungkinan kita bisa drop *missing value* pada 2 kolom ini (kita akan cek lebih lanjut)

Setelah melihat gambaran kasar dari *missing value* pada data yang kita miliki, selanjutnya kita akan menangani *missing value* dengan dua pendekatan :

* pertama : menghapus baris/kolom yang kosong. Cara ini tidak di sarankan untuk untuk beberapa kolom. Contoh pada kolom `Disposition` memiliki 35.69% *missing value* yang tergolong cukup tinggi sehingga kita akan kehilangan cukup banyak data jika di drop/hapus. Tapi jika dilihat pada kolom `Date Received` (0.128%) dan `Claim Site` (0.497%) bisa dibilang memiliki proporsi yang kecil, jadi ada kemungkinan bisa kita hapus baris yang mengandung *missing value* pada kedua kolom ini.

* kedua : metode yang lebih disarankan yaitu mengisi data yang hilang atau *missing value*. Ada beberapa cara/metode untuk mengisi nilai kosong ini yaitu dengan mengisi data yang hilang berdasarkan kolom lain yang secara *domain Knowledge* atau statistik memiliki hubungan dengan kolom yang memiliki nilai kosong ini. Setelah menggunakan metode ini dan ternyata masih ada nilai yang kosong maka kita bisa mengisi dengan angka *mean, median* atau *modus*. Menghapus data akan menjadi pilihan terakhir (jika sudah tidak ada cara lagi untuk mengisi nilai kosong ini).

#### `Status`, `Disposition`, `Claim Amount` dan `Close Amount`

Dari penjabaran sebelumnya *missing value* pada kolom `Status`, `Disposition`, `Claim Amount` dan `Close Amount` saling berkaitan, jadi kita akan isi *missing value* keempat kolom ini secara bersamaan

In [ ]:
df[df['Disposition'].isnull()]['Status'].value_counts()

##### Pertama, kita isi *missing value* pada kolom `Claim Amount`, `Close Amount` dan `Disposition` sesuai dengan kolom `Status` dengan aturan sebagai berikut :

* kasus pertama : Jika status nya *Approved*, *Approve in Full* dan nilai kolom `Disposition` nya kosong \
maka kita isi *missing value* `Close Amount` = nilai `Claim Amount` (dan sebaliknya) dan isi *missing value* `Disposition` = *Approve in Full*

In [ ]:
df1 = df[((df['Status']=="Approved") | (df['Status']=="Approve in Full")) & ((df['Disposition'].isnull()))]
idx_df1 = df1.index

#Isi nilai kosong pada kolom `Disposition` dengan "Approve in Full"
df.loc[idx_df1, "Disposition"] = "Approve in Full"

In [ ]:
#isi missing value pada kolom Claim Amount dan Close Amount
for idx in df[df["Disposition"] == "Approve in Full"].index:
    if (df['Claim Amount'][idx] == np.nan) and (df['Close Amount'][idx] != np.nan) :
        df.loc[idx, "Claim Amount"] = df[df['Close Amount']][idx]
    elif (df['Claim Amount'][idx] != np.nan) and (df['Close Amount'][idx] == np.nan) :
        df.loc[idx, "Close Amount"] = df[df['Claim Amount']][idx]

* kasus kedua : Jika statusnya *Deny*, *Denied* dan nilai kolom `Disposition` nya kosong \
maka kita isi *missing value* `Close Amount` = 0 dan `Claim Amount` = 0 serta isi *missing value* `Disposition` = *Deny*

In [ ]:
df2 = df[((df['Status']=="Deny") | (df['Status']=="Denied")) & (df['Disposition'].isnull())]
idx_df2 = df2.index

#Isi nilai kosong pada kolom `Disposition` dengan "Deny"
df.loc[idx_df2, "Disposition"] = "Deny"

In [ ]:
df.loc[df['Disposition']=="Deny", "Close Amount"] = df.loc[df['Disposition']=="Deny", "Close Amount"].fillna(0)
df.loc[df['Disposition']=="Deny", "Claim Amount"] = df.loc[df['Disposition']=="Deny", "Claim Amount"].fillna(0)

* kasus ketiga : status *Canceled* artinya klaim tersebut telah dibatalkan oleh pengaju/penumpang sehingga bisa kita kategorikan klaim ini sebagai kasus yang sudah "terselesaikan" antara penumpang dengan pihak TSA. Jadi kita akan masukkan status *Canceled* ke penyelesaian akhir (`Disposition`) *Settle*. Selain itu kita asumsikan juga klaim *Canceled* artinya nilai `Claim Amount` dan `Close Amount` sama dengan 0, karena memaang klaim ini telah terselesaikan (dengan cara pembatalan klaim oleh persetujuan pihak TSA) tanpa adanya pengajuan/pengembalian dana.

In [ ]:
df.loc[df['Status']=="Canceled", "Close Amount"] = df.loc[df['Status']=="Canceled", "Close Amount"].fillna(0)
df.loc[df['Status']=="Canceled", "Disposition"] = df.loc[df['Status']=="Canceled", "Disposition"].fillna("Settle")
df.loc[df['Status']=="Canceled", "Claim Amount"] = df.loc[df['Status']=="Canceled", "Claim Amount"].fillna(0)

* kasus keempat : ketika status nya *Settle*, *Seetled*\
maka kita isi *missing value* `Disposition` = "Settle"

In [ ]:
df.loc[(df['Status']=="Settle") | (df['Status']=="Settled"), "Disposition"] = df.loc[(df['Status']=="Settle") | (df['Status']=="Settled"), "Disposition"].fillna("Settle")

--> pada status *Settle* artinya klaim disetujui dan dana yang diberikan (Close Amount) tidak penuh atau tidak seratus persen dari dana klaim yang diajukan (Claim Amount) maka kita akan mengisi nilai kosong pada kolom `Close Amount` dengan mencari rata-rata proporsi/persentase dari dana klaim yang diajukan (Claim Amount) yang akan diteruskan ke pengaju klaim (Close Amount) jika nilai `Disposition` nya *Settle*

In [ ]:
df_amount_percentage = df[df['Disposition']=="Settle"][["Claim Amount", "Close Amount"]]

df_amount_percentage['Amount percentage'] = df["Close Amount"]/df["Claim Amount"]
df_amount_percentage.head()

In [ ]:
fig = px.histogram(df_amount_percentage["Amount percentage"], range_x=(0,10))
fig.add_annotation(text=f"Median Value : {df_amount_percentage['Amount percentage'].median()}", x=4, y=6000, arrowside="none")

Jika kita lihat dari gambar di atas, kebanyakan dana yang diberikan (Close Amount) tepat 0.5 (nilai median juga) atau 50% dari dana yang diajukan (Claim Amount), maka kita akan isi *missing value* pada kolom `Close Amount` dengan 0.5*(nilai `Claim Amount`)

Begitu juga sebalik nya ketika nilai dari kolom `Claim Amount` memiliki nilai NaN tapi kolom `Close Amount` nya tidak kosong pada status/disposition *settle* maka \
kita akan isi *missing value* pada kolom `Claim Amount` = 2*(nilai `Close Amount`)

In [ ]:
for idx in df[df['Disposition']=="Settle"].index:   
    if (df['Claim Amount'][idx] == np.nan) and (df['Close Amount'][idx] != np.nan):
        df.loc[df['Disposition']=="Settle", "Close Amount"][idx] = 0.5*(df.loc[df['Status']=="Settle", "Claim Amount"][idx])
    elif (df['Close Amount'][idx] == np.nan) and (df['Claim Amount'][idx] != np.nan):
        df.loc[df['Disposition']=="Settle", "Claim Amount"][idx] = 2*(df.loc[df['Status']=="Settle", "Close Amount"][idx])

Mari kita lihat status yang lainnya

* kasus kelima : ketika status nya *Closed as a contractor claim* \ jika dilihat dari definisi yang saya temukan di internet(https://www.lawinsider.com/dictionary/contractor-claim) artinya klaim yang diajukan sesuai dengan Kontrak Konstruksi untuk barang atau jasa yang disediakan berdasarkan atau sesuai dengan Kontrak Konstruksi. Jika dilihat dari value count dari kolom `Close Amount` semua nya bernilai 0 yang ini artinya kita bisa asumsikan bahwa semua klaim dengan status *Closed as a contractor claim* ditolak oleh TSA sehingga kita akan isi *missing value* pada kolom `Close Amount`, `Claim Amount` dengan 0 dan *missing value* pada kolom `Disposition` dengan *Deny* (sesuai definisi kita yaitu ketika nilai `Close Amount`== 0 maka `Disposition` = Deny)

In [ ]:
df[df["Status"]=="Closed as a contractor claim"]['Close Amount'].value_counts(dropna=False)


In [ ]:
df.loc[df["Status"]=="Closed as a contractor claim", "Close Amount"] = df.loc[df["Status"]=="Closed as a contractor claim", "Close Amount"].fillna(0)
df.loc[df["Status"]=="Closed as a contractor claim", "Disposition"] = df.loc[df["Status"]=="Closed as a contractor claim", "Disposition"].fillna("Deny")
df.loc[df["Status"]=="Closed as a contractor claim", "Claim Amount"] = df.loc[df["Status"]=="Closed as a contractor claim", "Claim Amount"].fillna(0)

* kasus keenam : status *Insufficient; one of the following items required: sum certain; statement of fact; signature; location of incident; and date.* atau kita sebut saja *Insufficient* artinya bahwa syarat dari klaim tidak terpenuhi maka kita bisa asumsikan *final settlement* atau `Disposition` dari status ini adalah *Deny*. Selain itu kita juga bisa mengisi *missing value* pada kolom `Claim Amount` dan `Close Amount` dengan nilai 0.

In [ ]:
#kita ubah nama nya dulu karena terlalu panjang
df.loc[df['Status']=="Insufficient; one of the following items required: sum certain; statement of fact; signature; location of incident; and date.", "Status"] = "Insufficient"

In [ ]:
df[df['Status']=="Insufficient"]["Disposition"].value_counts(dropna=False)

Sebelum kita isi nilai *missing value* kita lihat bahwa ada 5 baris dimana statusnya *Insufficient* tapi nilai `Disposition` nya *Approve in Full*. Ada 2 kemungkinan untuk kondisi ini. Pertama bisa saja final settlement atau penyelesaian/keputusan akhir dari klaim ini sebenar nya sudah *closed* tapi status nya belum diubah karena kolom `Status` disini bersifat keputusan sementara. Kedua memang ada kesalahan input data pada kolom `Disposition`. Disini saya akan ambil kemungkinan pertama, karena biasanya kolom `Disposition` atau keputusan akhir lah yang lebih diutamakan untuk digunakan jika ada kondisi seperti ini.

In [ ]:
df.loc[(df['Status']=="Insufficient") & (df['Disposition']=="Approve in Full"), "Status"] = "Approve in Full"

selanjutnya baru kita isi **missing value* pada kolom `Disposition` dari status ini dengan *Deny* dan mengisi *missing value* pada kolom `Claim Amount` dan `Close Amount` dengan nilai 0.

In [ ]:
df.loc[df['Status']=="Insufficient", "Disposition"] = df.loc[df['Status']=="Insufficient", "Disposition"].fillna("Deny")
df.loc[(df['Status']=="Insufficient") & (df['Claim Amount'].isnull()), "Claim Amount"] = df.loc[(df['Status']=="Insufficient") & (df['Claim Amount'].isnull()), "Claim Amount"].fillna(0)
df.loc[(df['Status']=="Insufficient") & (df['Close Amount'].isnull()), "Close Amount"] = df.loc[(df['Status']=="Insufficient") & (df['Close Amount'].isnull()), "Close Amount"].fillna(0)

* kasus ketujuh/terakhir : status *In litigation*, *Claim entered*, *In review* dan *Claim has been assigned for further investigation* bisa kita drop baris yanng mengandung nilai status ini. Alasan pertama : jumlah data nya sangat sedikit. Alasan kedua : keempat status tersebut bisa dibilang sedang dalam masa ketidakpastian, jadi bisa kita drop saja dari dataset karena bisa kita anggap juga sebagai "outliers" yang mengganggu analisis


In [ ]:
df[df['Disposition'].isnull()]['Status'].value_counts()

In [ ]:
idx_to_drop_in_status = [idx for idx in df.index if df['Status'][idx] in ["In litigation", "Claim entered", "In review" , "Claim has been assigned for further investigation"]]
df = df.drop(idx_to_drop_in_status, axis=0)

Mari kita cek berapa *missing value* yang tersisa

In [ ]:
#persentase missing value setiap kolom
missing_pct_df = pd.DataFrame({"column name" : df.columns.tolist(), "percentage (missing value)" : [df[col].isnull().sum()*100/len(df) for col in df.columns]}).sort_values(by="percentage (missing value)", ascending=False)
missing_pct_df

##### Kedua, kita akan coba lihat nilai kosong pada kolom `Claim Amount` jika dikaitkan dengan kolom `Claim Type`

In [ ]:
df_copy = df.copy()

In [ ]:
#Buat kolom baru, apakah null atau tidak pada kolom `Claim Amount
df_copy.loc[df_copy['Claim Amount'].isnull(), 'is_ClaimAmount_null'] = "null"
df_copy.loc[df_copy['Claim Amount'].notnull(), 'is_ClaimAmount_null'] = "not null"

cross tabulasi antara kolom baru `is_claim_amount_null` dengan `Claim Amount` dan `is_close_amount_nul`' dengan `Close Amount`

In [ ]:
pd.crosstab(df['Claim Type'], df_copy['is_ClaimAmount_null'], normalize='index')

untuk kolom `Claim Amount`, nilai kosong banyak berada pada tipe klaim nya berupa non-material seperti *Complaint*, *Compliment*, *Wrongful Death* dan *Passenger Theft* karena sulit rasanya untuk mengukur besaran klaim yang diajukan. Kita akan sesuaikan dengan kolom `Claim Type` , karena biasanya besaran dana klaim yang diajukan sesuai dengan jenis klaim/kategori item nya

1) pada tipe klaim *Complaint* kita akan hapus baris yang berisi *missing value* pada kolom `Claim Amount` karena proporsinya cukup tinggi yaitu >70% dari data total tipe klaim *Complaint*, tidak ada cara yang tepat untuk mengisi nilai kosong ini. Total ada 34 baris yang kita drop di kasus ini

In [ ]:
len(df.loc[(df['Claim Type']=="Complaint") & (df['Claim Amount'].isnull())])

In [ ]:
df = df.drop(df[df['Claim Type']=="Complaint"].loc[df['Claim Amount'].isnull()].index, axis=0)

2. kita juga akan menghapus baris yang memiliki tipe klaim "Compliment" karena kurang relevan untuk analisis. Kita hanya akan memproses klaim yang bersifat merugikan passenger

In [ ]:
df[df['Claim Type']=="Compliment"]

In [ ]:
df = df.drop(df[df['Claim Type']=="Compliment"].index, axis=0)

3. tipe klaim "Employee Loss (MPCECA)" 

In [ ]:
px.histogram(df[df['Claim Type']=="Employee Loss (MPCECA)"]['Claim Amount'], width=1000)

distribusi `Claim Amount` terlihat condong ke kanan atau tidak normal pada tipe klaim "Employee Loss (MPCECA)" , jadi kita akan isi nilai kosong dengan nilai median

In [ ]:
median1 = df[df['Claim Type']=="Employee Loss (MPCECA)"]['Claim Amount'].median()
median1

In [ ]:
df.loc[df['Claim Type']=="Employee Loss (MPCECA)", "Claim Amount"] = df.loc[df['Claim Type']=="Employee Loss (MPCECA)", "Claim Amount"].fillna(median1)

4. tipe klaim *Motor Vehicle*

In [ ]:
df[df['Claim Type']=="Motor Vehicle"]['Claim Amount']

In [ ]:
px.histogram(df[df['Claim Type']=="Motor Vehicle"]['Claim Amount'], width=1000)

distribusi `Claim Amount` juga terlihat condong ke kanan atau tidak normal pada tipe klaim "Motor Vehicle" , jadi kita akan isi nilai kosong dengan nilai median

In [ ]:
median2 = df[df['Claim Type']=="Motor Vehicle"]['Claim Amount'].median()
median2

In [ ]:
df.loc[df['Claim Type']=="Motor Vehicle", "Claim Amount"] = df.loc[df['Claim Type']=="Motor Vehicle", "Claim Amount"].fillna(median2)

5. tipe klaim *Personal Injury*

In [ ]:
px.box(df[df['Claim Type']=="Personal Injury"]['Claim Amount'], width=1000)

Bisa terlihat masih ada 57 baris yang nilai Claim Amount nya cukup tinggi hingga mencapai > 1 juta dollar yang bisa kita anggap sebagai outlier. 

In [ ]:
len(df[(df['Claim Type']=="Personal Injury") & (df['Claim Amount'] >= 1000000)])

In [ ]:
df[df['Claim Type']=="Personal Injury"].isnull().sum()

In [ ]:
print(df[df['Claim Type']=="Personal Injury"]['Claim Amount'].max(), df[df['Claim Type']=="Personal Injury"]['Claim Amount'].min())

Sebaran `Claim Amount` pada tipe klaim *Personal Injury* terlihat sangat bervariasi, ada nilai minimumnya $0 dan nilai maksimum nya $3 Billion. Akan sulit mengisi *missing value* `Claim Amount` pada kondisi ini kita akan drop saja.

In [ ]:
idx_to_drop_personal_injury =  df.loc[(df['Claim Type']=="Personal Injury") & (df['Claim Amount'].isnull())].index
df = df.drop(idx_to_drop_personal_injury, axis=0)

6. tipe klaim *Passenger Property Loss*

In [ ]:
px.box(df[df['Claim Type']=="Passenger Property Loss"]['Claim Amount'], width=1000)

In [ ]:
df_temp = df[df['Claim Type']=="Passenger Property Loss"]
df_temp[df_temp['Claim Type']=="Passenger Property Loss"].isnull().sum()

In [ ]:
df_temp[df_temp[['Claim Amount', 'Close Amount', 'Status', 'Disposition']].isnull().all(axis=1)]

Sebaran `Claim Amount` pada tipe klaim *Passenger Property Loss* terlihat sangat bervariasi, selain itu hampir semua nilai pada kolom `Claim Amount`, `Close Amount`, `Status`, `Disposition` bernilai NaN atau kosong, sehingga tidak ada cara yang tepat untuk mengisi *missing value* ini, mari kita drop saja

In [ ]:
idx_to_drop_property_loss =  df.loc[(df['Claim Type']=="Passenger Property Loss") & (df['Claim Amount'].isnull())].index
df = df.drop(idx_to_drop_property_loss, axis=0)

7. tipe klaim *Property Damage*

In [ ]:
px.box(df[df['Claim Type']=="Property Damage"]['Claim Amount'], width=1000)

In [ ]:
df[(df['Claim Type']=="Property Damage") & (df['Claim Amount'] >= 300000)]

Jika dilihat dari gambar di atas, maka ada 3 baris data dengan nilai `Claim Amount` sebagai outlier 

In [ ]:
df[df['Claim Type']=="Property Damage"].isnull().sum()

In [ ]:
df_temp = df[df['Claim Type']=="Property Damage"]
df_temp[df_temp[['Claim Amount', 'Close Amount', 'Status', 'Disposition']].isnull().all(axis=1)]

Kita akan drop 4206 baris ini, karena nilai pada kolom `Claim Amount`, `Close Amount`, `Status`, `Disposition` bernilai NaN atau kosong, sehingga tidak ada cara yang tepat untuk mengisi *missing value* ini

In [ ]:
df = df.drop(df_temp[df_temp[['Claim Amount', 'Close Amount', 'Status', 'Disposition']].isnull().all(axis=1)].index, axis=0)

Sebelum lanjut, mari kita isi dulu nilai `Close Amount` yang kosong, jika nilai `Claim Amount` dan `Disposition` nya tidak kosong

In [ ]:
df[(df['Close Amount'].isnull()) & ((df['Claim Amount'].notnull()) & (df['Disposition'].notnull()))]

In [ ]:
df.loc[(df['Close Amount'].isnull()) & (df['Claim Amount'].notnull()), 'Close Amount'] = df.loc[(df['Close Amount'].isnull()) & (df['Claim Amount'].notnull()), 'Claim Amount']

Kita juga akan drop sisa baris (205 baris) yang memiliki *missing value* pada kolom `Claim Amount`, `Close Amount`, `Status`, `Disposition`

In [ ]:
df[df[['Claim Amount', 'Close Amount', 'Status', 'Disposition']].isnull().all(axis=1)]

In [ ]:
df = df.drop(df[df[['Claim Amount', 'Close Amount', 'Status', 'Disposition']].isnull().all(axis=1)].index, axis=0)

Mari kita cek kembali *Missing Value* yang tersisa

In [ ]:
#persentase missing value setiap kolom
missing_pct_df = pd.DataFrame({"column name" : df.columns.tolist(), "percentage (missing value)" : [df[col].isnull().sum()*100/len(df) for col in df.columns]}).sort_values(by="percentage (missing value)", ascending=False)
missing_pct_df

#### `Incident Date` dan `Date Received`

pada kolom `Incident Date` dan `Date Received` ada nilai *missing value* yang diwakilkan oleh teks "nan", kita akan convert dulu ke numpy.NaN

In [ ]:
df.loc[df['Date Received']=="nan", "Date Received"] = np.nan

In [ ]:
df.loc[df['Incident Date']=="nan", "Incident Date"] = np.nan

In [ ]:
df[df['Incident Date'].isnull()]

Jika kita lihat beberapa baris memiliki *missing value* pada kolom `Incident Date` tapi nilai pada kolom `Date Received` nya tidak kosong. \
Maka bisa kita asumsikan bahwa waktu antara kejadian dan klaim terjadi di hari yang sama, kita akan isi *misiing value* ini dengan data tanggal kolom `Date Received`

In [ ]:
df['Incident Date'] = df['Incident Date'].astype(str).replace('NaT', np.nan)
df['Date Received'] = df['Date Received'].astype(str).replace('NaT', np.nan)

In [ ]:
df['Date Received'] = pd.to_datetime(df['Date Received'])
df['Incident Date'] = pd.to_datetime(df['Incident Date'])

In [ ]:
for idx in df[df['Incident Date'].isnull()].index:
    if df['Date Received'][idx] != np.nan:
        df.loc[idx, 'Incident Date'] = df['Date Received'][idx] 

Mari kita cek *missing value* pada kolom `Date Received`, jika ada kita akan isi dengan data tanggal kolom `Incident Date`

In [ ]:
df[df['Date Received'].isnull()]

In [ ]:
for idx in df[df['Date Received'].isnull()].index:
    if df['Incident Date'][idx] != np.nan:
        df.loc[idx, 'Date Received'] = df['Incident Date'][idx] 

#### `Airline Name`, `Airport Code`, dan `Airport Name`

Mari kita cek dulu antara kolom `Airport Name` dan ` Airport Code`karena keduanya saling berkaitan (lebih tepatnya memang dua entitas yang sama)

Kita cek jika salah satunya kosong

In [ ]:
df[(df['Airport Name'].isnull()) & (df['Airport Code'].notnull())]

In [ ]:
df[(df['Airport Name'].notnull()) & (df['Airport Code'].isnull())]

Jadi sudah dipastikan pasangan nya lengkap, tidak ada yang salah satunya kosong

In [ ]:
df[df["Airport Name"].isnull()]

Kita akan buat nama/kategori baru pada kolom `Airport Name` dan ` Airport Code` jika keduanya kosong. Kita akan sebut dengan *Unknown* karena tidak ada cara yang tepat untuk mengisi kolom ini.

In [ ]:
df.loc[:, ['Airport Name', 'Airport Code']] = df.loc[:, ['Airport Name', 'Airport Code']].fillna("Unknown")

Ada dua cara untuk mengisi nilai *missing value* pada kolom `Airline Name` :

* Pertama : kita mengisi dengan nilai modus `Airline Name` berdasarkan `Airport Name`
* Kedua : buat kategori/nama baru dengan sebutan *Unknown*

kelemahan cara pertama, kita seolah-olah asala "menebak" maskapai mana yang di komplain oleh penumpang. Akibatnya terjadi penurunan rating terhadap maskapai tersebut yang padahal belum tentu kejadian terjadi di maskapai itu. Maka untuk menghindari kesalahpahaman, kita akan gunakan cara kedua

In [ ]:
df.loc[:, 'Airline Name'] = df.loc[:, 'Airline Name'].fillna("Unknown")

#### `Claim Type`, `Item`, dan `Claim Site`

Mari kita cek persentase *Missing Value* yang tersisa

In [ ]:
#persentase missing value setiap kolom
missing_pct_df = pd.DataFrame({"column name" : df.columns.tolist(), "percentage (missing value)" : [df[col].isnull().sum()*100/len(df) for col in df.columns]}).sort_values(by="percentage (missing value)", ascending=False)
missing_pct_df

In [ ]:
df[df['Claim Type'].isnull()]

Kolom `Item` dan `Claim Type` saling berhubungan karena biasanya sebuah item  masuk ke dalam kategori `Claim Type` tertentu. mari kita lihat satu sampel

In [ ]:
df[df['Item']=="Eyeglasses - (including contact lenses)"]['Claim Type'].value_counts()

Jadi pada item *Eyeglasses - (including contact lenses)* nilai modus dari tipe klaim nya yaitu *Passenger Property Loss*

Strateginya kita akan gunakan nilai modus dari `Claim Type` berdasarkan kolom `Item` 

In [ ]:
item_mode_df = df.groupby("Item")['Claim Type'].agg(lambda x: pd.Series.mode(x)[0] if (len(pd.Series.mode(x))>0) else pd.Series.mode(x)).reset_index(name="Mode")
item_mode_df

In [ ]:
for idx in df[df['Claim Type'].isnull()].index :
    if (df['Item'][idx] != np.nan) & (df['Item'][idx] in item_mode_df['Item'].values):
        try :
            df.loc[idx, 'Claim Type'] = item_mode_df[item_mode_df['Item']==df.loc[idx, "Item"]]['Mode'].iloc[0]
        except:
            continue

Selanjutnya dengan cara yang sama, kita akan mengisi *missing value* kolom `Item` dengan modus `Item` berdasarkan kolom `Claim Type`

In [ ]:
df[df['Item'].isnull()]

In [ ]:
claim_type_mode_df = df.groupby("Claim Type")['Item'].agg(lambda x: pd.Series.mode(x)[0] if (len(pd.Series.mode(x))>0) else pd.Series.mode(x)).reset_index(name="Mode")
claim_type_mode_df

In [ ]:
for idx in df[df['Item'].isnull()].index :
    if (df['Claim Type'][idx] != np.nan) & (df['Claim Type'][idx] in item_mode_df['Item'].values):
            df.loc[idx, 'Item'] = item_mode_df[item_mode_df['Claim Type']==df.loc[idx, "Claim Type"]]['Mode'].iloc[0]
        

sisanya kita akan isi *missing value* pada kolom `Item` dan `Claim Type` dengan nilai *Other*

In [ ]:
df.loc[df[(df['Item'].isnull())].index, 'Item'] = "Other"
df.loc[df[(df['Claim Type'].isnull())].index, 'Claim Type'] = "Other"

Untuk *missing value* pada kolom `Claim Site` kita isi dengan *Other*

In [ ]:
df.loc[df['Claim Site'].isnull(), 'Claim Site'] = "Other"

#### Lain-lain

kita cek kembali persentase **missing value**

In [ ]:
#persentase missing value setiap kolom
missing_pct_df = pd.DataFrame({"column name" : df.columns.tolist(), "percentage (missing value)" : [df[col].isnull().sum()*100/len(df) for col in df.columns]}).sort_values(by="percentage (missing value)", ascending=False)
missing_pct_df

In [ ]:
df[df['Claim Amount'].isnull()].sample(10)

In [ ]:
df[df['Claim Amount'].isnull()]['Disposition'].value_counts()

Jika kita lihat sepertinya ada beberapa nilai *missing value* pada kolom `Claim Amount` yang tertinggal untuk diisi. Kita akan gunakan definisi sebelumnya tentang `Disposition` dan `Close Amount` untuk mengisi nilai yang kosong ini

* Jika `Disposition` = *Approve in Full* maka *missing value* kolom `Claim Amount` = nilai `Close Amount`
* Jika `Disposition` = *Settle* maka *missing value* kolom `Claim Amount` = 2*(nilai `Close Amount`')

In [ ]:
df.loc[(df['Disposition']=="Approve in Full") & df['Claim Amount'].isnull(), 'Claim Amount'] = df.loc[(df['Disposition']=="Approve in Full") & df['Claim Amount'].isnull(), 'Close Amount']

In [ ]:
df.loc[(df['Disposition']=="Settle") & df['Claim Amount'].isnull(), 'Claim Amount'] = 2*(df.loc[(df['Disposition']=="Settle") & df['Claim Amount'].isnull(), 'Close Amount'])

In [ ]:
#persentase missing value setiap kolom
missing_pct_df = pd.DataFrame({"column name" : df.columns.tolist(), "percentage (missing value)" : [df[col].isnull().sum()*100/len(df) for col in df.columns]}).sort_values(by="percentage (missing value)", ascending=False)
missing_pct_df

Sisa *Missing Value* akan di drop karena proporsinya kecil dan juga tidak ada strategi lain untuk mengisi ini, maka kita bisa drop semua baris ini

In [ ]:
df = df.dropna(axis=0)

Untuk bahan analisis selanjutnya kolom `Status` akan kita drop, karena kolom tersebut sebenar nya sudah diwakilkan dengan kolom `Disposition` yang sudah kita definisikan di atas. Selain itu juga kita akan drop kolom `Claim Number` dan `Airport Code` (karena sudah diwakilkan oleh kolom `Airport Name`)

In [ ]:
df = df.drop(['Status', 'Claim Number', 'Airport Code'], axis=1)

In [ ]:
a = df.copy()

In [ ]:
df['Date Received'] = pd.to_datetime(df['Date Received'])
df['Incident Date'] = pd.to_datetime(df['Incident Date'])

### Catatan Tambahan

In [ ]:
df['different_day'] = (pd.to_datetime(df['Date Received']) - pd.to_datetime(df['Incident Date'])).dt.days
df[df['different_day'] <0]

Saat kita mencoba melihat perbedaan waktu antara `Date Received` dan `Incident Date`, ada 1595 baris data yang dimana waktu pelaporan klaim nya diterima sebelum kejadian terjadi. Secara logika jelas ini tidak mungkin. Ada beberapa kemungkinan hal ini bisa terjadi. Pertama : memang ada kesalahan input data karena biasanya beberapa orang salah menginput tanggal yang seharusnya tanggal kejadian malah dimasukkan ke kolom tanggal pelaporan atau salah ketik. Kedua : Bisa saja ini adalah "laporan fiktif" sehingga ada kemungkinan adanya laporan palsu. Kita anggap 1595 baris ini sebagai *outliers* atau anomali yang dimana kita akan exclude atau drop semua baris ini untuk analisis selanjutnya.

In [ ]:
df = df.drop(df[df['different_day']<0]['Disposition'].index, axis=0)

## Data yang sudah bersih

In [ ]:
df_obj = df.select_dtypes(['object'])
df[df_obj.columns] = df_obj.apply(lambda x: x.str.strip())

In [ ]:
listItem = []
for col in df.columns :
    listItem.append([col, df[col].dtype, df[col].isna().sum(),
                    df[col].nunique(), list(df[col].drop_duplicates().sample(3).values)])

dfDesc_clean = pd.DataFrame(columns=['dataFeatures', 'dataType', 'null', 'unique', 'uniqueSample'],
                     data=listItem)
print(df.shape)
dfDesc_clean

Sebelum dibersihkan, kita memiliki **204,267** baris data, sekarang kita memiliki **189,591** baris. 

Sebanyak **12,963** baris data (**6.35%** dari data awal) yang kosong dihapus, dan sisa data kosong diisi dengan data yang dirasa relevan. Selain itu ada 1595 baris data yang di drop karena data yang dimana waktu pelaporan klaim nya diterima sebelum kejadian terjadi, karena terlalu banyak ketidakpastian alasan kenapa bisa seperti itu kita drop saja. Sehingga total baris yang kita drop sebanyak 14,588 baris (**7.14%** dari data awal)

# Data Analysis

Setelah dilakukan tahap *data cleaning*, sekarang kita akan melakukan analisis untuk mencari tahu **bagaimana karakteristik keputusan/penyelesaian akhir yang diambil terhadap klaim yang diajukan?**

Analisis dilakukan dengan membandingkan data klaim yang disetujui secara penuh (`Disposition` = *Approve in Full*), klaim yang terselesaikan dengan persetujuan kedua belah pihak (`Disposition` = *Settle*) dan klaim yang ditolak (`Disposition` = *Deny*) berdasarkan *feature* yang tersedia pada dataset yang kita miliki. (catatan : kolom `Disposition` sebagai *target*)

Mari kita lihat sebaran dari kolom `Disposition`

In [ ]:
df['Disposition'].value_counts().reset_index().rename({"index" : "Disposition", "Disposition" : "count"}, axis=1)

In [ ]:
display(df['Disposition'].value_counts())
px.bar(df['Disposition'].value_counts().reset_index().rename({"index" : "Disposition", "Disposition" : "count"}, axis=1), color='Disposition', color_discrete_sequence=['red', 'blue', 'green'] , width=600, opacity=0.7, text_auto=True
        , title="Distribusi kumlah klaim berdasarkan Disposition")

In [ ]:
df.head()

## Berdasarkan waktu kejadian dan pelaporan klaim : `Date Received` dan `Incident Date`

Pertama, mari kita lihat bagaimana pengaruh perbedaan waktu pelaporan klaim (Date Received) dengan tanggal kejadian (Incident date) dengan jumlah klaim dan keputusan akhir(*final settlement*). kita akan menggunakan kolom `Date Received` dan `Incident Date` untuk menjawab beberapa pertanyaan.

* Pada bulan apa jumlah klaim nya paling besar? Bagaimana komposisi status penyelesaian akhir (*final settlement*?) nya?
* Berapa selisih hari waktu pelaporan dan waktu kejadian yang memiliki paling banyak klaim nya? Bagaimana komposisi status penyelesaian akhir (*final settlement*?) nya?
* Apakah ada perbedaan yang signifikan antara selisih hari waktu pelaporan (`Date Received`) dan waktu kejadian (`Incident Date`) terhadap kelompok keputusan akhir (`Disposition`) klaim?

In [ ]:
df_month = df.copy()
df_month['Month_Date_Received'] = df_month['Date Received'].dt.month
fig = px.bar(df_month[['Month_Date_Received', 'Disposition']].value_counts().reset_index(name="count"),x='Month_Date_Received', y='count', title='Distribusi klaim berdasarkan bulan klaim diterima', width=1000, color='Disposition', color_discrete_sequence=['Red', 'Blue', 'Green'], opacity=0.7)
fig.update_xaxes(title="Month")
fig.show()


df_month['Month_Incident_Date'] = df_month['Incident Date'].dt.month
fig = px.bar(df_month[['Month_Incident_Date', 'Disposition']].value_counts().reset_index(name="count"),x='Month_Incident_Date', y='count', title='Distribusi klaim berdasrkan bulan Kejadian/Incident', width=1000, color='Disposition', color_discrete_sequence=['Red', 'Blue', 'Green'], opacity=0.7)
fig.update_xaxes(title="Month")
fig.show()

In [ ]:
df_ct1 = pd.crosstab(df_month['Month_Date_Received'], df['Disposition'])
df_ct1['Total'] = df_ct1.sum(axis=1)
display(df_ct1.sort_values(by='Total', ascending=False))


df_ct2 = pd.crosstab(df_month['Month_Incident_Date'], df['Disposition'])
df_ct2['Total'] = df_ct2.sum(axis=1)
display(df_ct2.sort_values(by='Total', ascending=False))

Dari hasil cross tabulasi dan grafik di atas  kita dapat melihat bahwa laporan klaim paling banyak terjadi di bulan januari dan kejadian/peristiwa terjadi cukup banyak di bulan desember. \
Ini masuk akal mengingat penerbangan paling sibuk di akhir tahun karena adanya liburan tahun baru, sehingga laporan baru masuk di bulan januari.\

Selain itu dapat kita lihat bahwa jumlah klaim tersebar cukup merata pada setiap bulan nya. Secara kasar kita dapat melihat bahwa hampir tidak ada pengaruh bulan laporan klaim terhadap kolom `Disposition`.

Sekarang kita perhatikan kolom `different_day` yang sudah kita definisikan di langkah cleaning. Dimana `different_day` adalah selisih antara tanggal masuk klaim dengan tanggal kejadian


In [ ]:
df[df['different_day'] > 365]['Disposition'].value_counts()

In [ ]:
px.histogram(df[df['different_day'] <= 365], x='different_day', color='Disposition', title='Distribusi different_day berdasarkan Disposition (Target) periode 1 tahun (365 hari)', color_discrete_sequence=['Blue', 'Green', 'Red'], opacity=0.7)

In [ ]:
px.histogram(df[df['different_day'] > 365], x='different_day', color='Disposition', title='Distribusi different_day berdasarkan Disposition (Target) periode > 365 hari', color_discrete_sequence=['Blue', 'Green', 'Red'], opacity=0.7)

In [ ]:
df[df['different_day'] > 365]['Disposition'].value_counts()

In [ ]:
print(df.shape)
df[(df['different_day'] == 11)]['Disposition'].value_counts()

In [ ]:
df.shape

Dari grafik di atas dapat dilihat bahwa :

* Ada 1061 baris data yang perbedaan tanggal masuk klaim dan pelaporan nya lebih dari 365 hari (> 1 tahun), dimana 1061 baris klaim nya ditolak (*Deny*) dan sisanya disetujui (*Approve in Full* : 373 baris dan *Settle* : 367 baris). Hal ini dikarenakan penumpang bisa mengajukan klaim ke TSA atas barang yang hilang/rusak sampai lebih dari 2 tahun. Bisa dibaca pada form pengisian klaim resmi TSA (https://www.tsa.gov/travel/passenger-support/claims#:~:text=Processing,a%20claim%20rests%20with%20TSA.)
* Waktu pelaporan paling banyak terjadi setelah 10-11 hari semenjak waktu kejadian yaitu ada 5359 baris dari total 189591 baris dengan komposisi : *Deny* (2951 baris), *Approve in Full* (1351 baris), dan *Settle* (1057 baris). Kenapa 11 hari? setelah saya baca pada form pengisian (link ada di atas) memang sering terjadi delay pengiriman surat yang dikirim ke fasilitas Federal karena adanya proses *screening* up to 3 minggu, jadi wajar saja klaim baru masuk setidaknya antara di hari ke-7 sampai ke-14. Jadi masuk akal paling banyak klaim masuk di hari ke-11

Sekarang, mari kita lihat, apakah perbedaan `different_day` antara ketiga kelompok final settlement (`Disposition`) signifikan atau tidak.


In [ ]:
# cek distribusi kolom different_day
from scipy.stats import normaltest
stats, pval=normaltest(df['different_day'])
if pval<=0.05:
    print('tidak normal') #Ha
else: 
    print('distribusi normal') #Ho 

In [ ]:
# uji statistik, apakah perbedaan `different_day` antara ketiga kelompok final settlement (`Disposition`) signifikan atau tidak.
# karena distribusi data tidak normal, Kruskal-Wallis
from scipy.stats import kruskal
stats,pval=kruskal(*[df[df['Disposition']==i]['different_day'] for i in df['Disposition'].unique()])
if pval <= 0.05:
    print('Tolak Ho')
else:
    print('Gagal menolak Ho')

Dengan ini Kita memiliki bukti yang cukup untuk menyimpulkan bahwa `different_day` pada klaim menyebabkan perbedaan keputusan akhir (`Disposition`) yang signifikan secara statistik.

## Berdasarkan dana pengajuan klaim dan dana kompensasi : `Claim Amount` dan `Close Amount`

Mari kita lihat distribusi `Claim Amount` berdasarkan kolom `Disposition`

In [ ]:
px.histogram(df[df['Claim Amount']<100], 'Claim Amount' ,color='Disposition')

Jika kita lihat bahwa distribusi `Claim Amount` bisa dibilang *extremely positively skewed* atau sangat condong ke kanan, yang artinya nilai nya tersebar terpusat pada nilai `Claim Amount` di sebelah kiri. Untuk itu, strategi yang akan saya gunakan yaitu *binning* atau *discretization* untuk mengubah nilai kontinu menjadi data ordinal, sehingga data `Claim Amount` sebaran nya menjadi lebih seimbang. Disini saya akan  menggunakan pandas.qcut() atau *quantile-based discretization* membagi data dengan ukuran bucket yang sama berdasarkan peringkat atau beberapa kuantil sampel, disini saya akan bagi data menjadi 5 bin sehingga kita akan memiliki 5 kategori.

In [ ]:
pd.cut(df['Claim Amount'], bins=[0,10,100,500,10000,3000000000000]).value_counts()

bisa dilihat dari hasil value_counts() di atas, kita mempunyai 4 bin atau 4 kategori yaitu :

* range `0 - $10` kita akan beri label **Very Small (0-$10)**
* range `$10 - $100` kita akan beri label **Small($10-$100)**
* range `$100 - $500`kita akan beri label **Medium($100-$500)**
* range `$500 - $100k` kita akan beri label **Large ($500-$100k)**
* range `$100k-$300B` kita akan beri label **Very Large ($100k-$300B)**

In [ ]:
df['Claim_Amount_category'] = pd.qcut(df['Claim Amount'], 5, labels=["Very Small (0-$10)", "Small ($10-$100)", "Medium ($100-$500)","Large ($500-$100k)", "Very Large ($100k-$300B)"])
df[['Claim Amount', 'Claim_Amount_category']]

Kita akan coba membuat cross tabulasi antara kolom `Claim_Amount_category` dan `Disposition`

In [ ]:
df_ct3 = pd.crosstab(df['Claim_Amount_category'], df['Disposition'])
df_ct3['Total'] = df_ct3.sum(axis=1)
df_ct3

In [ ]:
px.imshow(df_ct3.iloc[:,0:3], width=600, color_continuous_scale='blues', aspect='auto', height=600)

In [ ]:
df[df['Claim Amount']> 10000000][['Claim Amount', 'Claim_Amount_category', 'Disposition']]

Dari hasil cross tabulasi dan grafik heat map di atas kita dapat melihat beberapa hal :

* Klaim yang di setujui secara penuh (*Approve in Full*) paling banyak saat dana klaim (`Claim Amount`) yang diajukan memiliki angka uang cukup kecil ($10-$100). Tentu saja ini masuk akal, karena jika angka dana klaim nya kecil kemungkinan besar akan disetujui dan dana yang diberikan seratus persen.
* Klaim yang ditolak paling banyak ada di kategori claim amount yang sangat kecil (very small), kemungkinan memang banyak ada nya laporan fiktif/palsu yang masuk karena angka klaim nya kecil sehingga ketidakpastian tentang keaslian klaim tinggi. Klaim yang ditolak kedua paling banyak yaitu berada di kategori Very Large ($100k-$300B), hal ini masuk akal karena angkanya yang dibilang cukup fantastis sehingga butuh ketelitian pihak TSA dalam memfilter klaim yang mana yg harus disetujui apalagi kalau sudah berbicara tentang budget untuk memberikan dana kompensasi. Kita ambil 11 sampel dimana besar `Claim Amount` lebih besar dari 10 juta dollar ($10,000,000) yang bisa dilihat dari tabel di atas, dimana 9 dari 11 sampel ajuan klaim nya ditolak. 
* Tiga `Claim_Amount_category` teratas dari klaim dengan status final settlement (Disposition) Settle : yaitu Medium ($100-$500), Large ($500-$100k) dan Very Large ($100k-$300B). Artinya negosiasi antara pihak TSA dan penggugat/pengaju klaim biasanya terjadi pada kasus dengan nilai klaim di sekitar angka ini karena angkanya yang cukup besar dengan harapan kesepakatan terjadi.

In [ ]:
df_ct3.drop('Total', axis=1, inplace=True)
df_ct3.columns = ["Approve in Full","Deny","Settle"]
df_ct3.index = ["Very Small (0-$10)", "Small($10-$100)", "Medium ($100-$500)","Large ($500-$100k)", "Very Large ($100k-$300B)"]

observed = df_ct3.iloc[0:5,0:3] 
observed

Selanjutnya kita akan melakukan uji statistik apakah proporsi dari setiap kelompok keputusan akhir (final settlement) yaitu `Disposition` memiliki hubungan yang signifikan dengan kelompok dana klaim yang diajukan (`Claim_Amount_category`)

In [ ]:
# uji statistik, apakah proporsi dari setiap kategori kolom `Disposition` memiliki hubungan yang signifikan dengan kategori kolom `Claim_Amount_category`
# karena distribusi data tidak normal, Chi-Squared test
from scipy.stats import chi2_contingency
stats,pvalue,*_=chi2_contingency(observed)
if pvalue <= 0.05:
    print('Tolak Ho')
else:
    print('Gagal menolak Ho')

Ternyata memang ada hubungan yang kuat/signifikan antara proporsi setiap kelompok pada kolm `Disposition` dengan kelompok dana yang diajukan (`Claim_Amount_category`)

Untuk kolom `Close Amount` secara *Domain Knowledge* memiliki hubungan yang kuat dengan `Claim Amount`, `Claim_Amount_category` dan `Disposition`. Karena besaran dana yang akan diberikan ke pneggugat/pengaju tergantung dari keputusan akhir atau final settlement/Disposition pihak TSA. Jadi jika keputusan nya sudah ada dan Claim amount nya sudah tertulis tinggal kita mengikuti aturan yang ada jika *Approve in Full* maka dana diberikan 100%, jika *Settle* maka dana yang diberikan dengan proporsi tertentu sesuai dengan kesepakatan dan jika *Deny* sudah pasti dana yang dikembalikan 0 dollar.

## Berdasarkan jenis benda/item yang di klaim : `Claim Type`

Mari kita lihat dulu distibusi `Dispostion` berdasarkan tipe klaim nya `Claim Type`

In [ ]:
fig = px.histogram(df, x='Claim Type', facet_row='Disposition', facet_col_wrap=3)
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
fig.show()

Jika kita lihat distribusi tipe klaim tidak seimbang (Imbalanced) antara *Passenger Property Loss*, *Property Damage* dan yang lainnya. Disini kita akan buat kategori yang lainnya ini (*Motor Vehicle*, *Personal Injury*, *Passenger Theft*, *Employee Loss (MPCECA*), *Other*, *Complaint*, *Bus Terminal* dan *Wrongful Death*) sebagai kategori baru yaitu *Non-Property* 

In [ ]:
df.loc[~df['Claim Type'].isin(['Passenger Property Loss', 'Property Damage']), 'Claim Type'] = "Non-Property"

In [ ]:
fig = px.histogram(df, x='Claim Type', color='Disposition', color_discrete_sequence=['blue', 'green', 'red'], opacity=0.7, title="Distribusi klaim berdasarkan Claim Type")
fig.show()

In [ ]:
df_ct4 = pd.crosstab(df['Claim Type'], df['Disposition'])
df_ct4['Total'] = df_ct4.sum(axis=1)
df_ct4

In [ ]:
px.imshow(df_ct4.iloc[:,0:3], width=600, color_continuous_scale='blues', aspect='auto', height=600)

Dari hasil cross tabulasi di atas dan grafik dapat kita lihat beberapa hal berikut :

* Mayoritas klaim berasal dari klaim yang bertipe properti/benda milik penumpang hilang (*Passenger Property Loss*)
* Dari tipe klaim Non-Property ternyata hanya `1338/2633` atau `50.8%` yang ditolak (menarik untuk ditelaah lebih dalam di lain kesempatan)
* Tipe klaim yang proporsi di setujui secara penuh (*Approve in Full*) paling banyak pada tipe klaim (*Property Damage*) yaitu `21674/7193` atau `30.18%`
* Tipe klaim yang proporsi di setujui secara partial/sebagian (*Settle*) paling banyak pada tipe klaim (*Non-Property*) yaitu `699/2633` atau `26.54%`
* Tipe klaim yang proporsi di tolak (*Deny*) paling banyak pada tipe klaim (*Non-Property*) yaitu `1338/2633`  atau `50.8%`

Selanjutnya kita akan melakukan uji statistik apakah proporsi dari setiap kelompok keputusan akhir (final settlement) yaitu `Disposition` memiliki hubungan yang signifikan dengan kelompok tipe klaim (`Claim Type`)

In [ ]:
df_ct4.drop('Total', axis=1, inplace=True)
df_ct4.columns = ["Approve in Full","Deny","Settle"]
df_ct4.index = ["Non-Property", "Passenger Property Loss", "Property Damage"]

observed = df_ct4.iloc[0:3,0:3] 
observed

In [ ]:
# uji statistik, apakah proporsi dari setiap kategori kolom `Disposition` memiliki hubungan yang signifikan dengan kategori kolom Claim Type`
# karena kita akan membandingkan dua variabel kategori kita akan gunakan Chi-Squared test
from scipy.stats import chi2_contingency
stats,pvalue,*_=chi2_contingency(observed)
if pvalue <= 0.05:
    print('Tolak Ho')
else:
    print('Gagal menolak Ho')

Ternyata memang ada hubungan yang kuat/signifikan antara proporsi setiap kelompok pada kolm `Disposition` dengan kelompok tipe klaim (`Claim Type`)

Untuk kolom `Item` akan di tunda dulu penjabaran nya karena butuh dirapikan lebih jauh datanya. Mari kita drop saja kolom ini

In [ ]:
df = df.drop('Item', axis=1)

## Berdasarkan tempat/lokasi klaim : `Airport Name`, `Airline Name`, dan `Claim Site` 

In [ ]:
# 10 bandara teratas dengan nilai klaim paling banyak
top_10_airport = df.groupby(['Airport Name'])['Disposition'].count().sort_values(ascending=False).iloc[:10].index.tolist()
top_10_airport

In [ ]:
top_10_airport_df = df[df['Airport Name'].isin(top_10_airport)].groupby(['Airport Name', 'Disposition']).size().reset_index(name='count')
for airport in top_10_airport:
    total = top_10_airport_df[top_10_airport_df['Airport Name']==airport]['count'].sum()
    top_10_airport_df.loc[top_10_airport_df['Airport Name']==airport, 'Total'] = total
top_10_airport_df['percentage'] = round(top_10_airport_df['count']*100/top_10_airport_df['Total'],2)
top_10_airport_df['percentage'] = top_10_airport_df['percentage'].astype(str) + "%" 
top_10_airport_df.sort_values(by='Total', ascending=False)

In [ ]:
px.bar(top_10_airport_df, x='Airport Name', y='count', color='Disposition', opacity=0.7, text='percentage', title="10 bandara teratas dengan nilai klaim paling banyak")

Dari hasil di atas bisa kita lihat beberapa hal berikut (disini kita akan mengabaikan nilai *Unknown*) :

* Jumlah klaim terbanyak ada pada bandara *Unknown* karena lumayan banyak porsi data yang kosong pada kolom `Airport Name`. Secara teknikal artinya bandara dengan jumlah klaim terbanyak jatuh pada Los Angeles International Airport yang dimana adalah bandara utama di kota Los Angeles dan salah satu bandara tersibuk ke-3 di USA [Los_Angeles_International_Airport](https://en.wikipedia.org/wiki/Los_Angeles_International_Airport#:~:text=In%202019%2C%20LAX%20handled%2088%2C068%2C013,Hartsfield%E2%80%93Jackson%20Atlanta%20International%20Airport.) dengan 8909 klaim. Tapi walaupun begitu proporsi klaim yang disetujui ketiga terbesar di antara yang 10 bandara lainnya.
* Bandara dengan **proporsi klaim yang ditolak terbesar** jatuh pada bandara McCarran International yaitu `66.41%`. Yang kedua adalah John F. Kennedy International yaitu `64.20%`, bandara ini juga memiliki jumlah klaim terbanyak kedua artinya bandara ini memiliki *Denied Claim Rate* (terhadap total klaim) yang tinggi di antara 10 bandara lainnya.
* Bandara dengan proporsi **klaim yang disetujui terbesar** (*Approve in Full* + *Settle*) jatuh pada bandara Seattle-Tacoma International dengan 30.28 + 23.46 = `53.74%`
* Bandara dengan proporsi **klaim yang disetujui secara penuh (*Approve in Full*) terbesar** jatuh pada bandara Seattle-Tacoma International dengan nilai `30.28%`
* Bandara dengan proporsi **klaim yang disetujui secara sebagian (*Settle*) terbesar** jatuh pada bandara Newark International Airport dengan nilai `25.85`

In [ ]:
# 10 airline teratas dengan nilai klaim paling banyak
top_10_Airline = df.groupby(['Airline Name'])['Disposition'].count().sort_values(ascending=False).iloc[:10].index.tolist()
top_10_Airline

In [ ]:
top_10_Airline_df = df[df['Airline Name'].isin(top_10_Airline)].groupby(['Airline Name', 'Disposition']).size().reset_index(name='count')
for Airline in top_10_Airline:
    total = top_10_Airline_df[top_10_Airline_df['Airline Name']==Airline]['count'].sum()
    top_10_Airline_df.loc[top_10_Airline_df['Airline Name']==Airline, 'Total'] = total
top_10_Airline_df['percentage'] = round(top_10_Airline_df['count']*100/top_10_Airline_df['Total'],2)
top_10_Airline_df['percentage'] = top_10_Airline_df['percentage'].astype(str) + "%" 
top_10_Airline_df.sort_values(by='Total', ascending=False)

In [ ]:
px.bar(top_10_Airline_df, x='Airline Name', y='count', color='Disposition', opacity=0.7, text='percentage', title="10 maskapai teratas dengan nilai klaim paling banyak")

Dari hasil di atas bisa kita lihat beberapa hal berikut (disini kita akan mengabaikan nilai *Unknown*) :

* Airline dengan jumlah klaim terbanyak jatuh pada Delta Airlines dengan total `23440` klaim dari total `189591` klaim
* Airline dengan **proporsi klaim yang ditolak terbanyak** jatuh pada Jet Blue yaitu `62.5%`. Yang kedua adalah Delta Airlines yaitu `57.58%`, Airline ini ini juga memiliki jumlah klaim yang paling banyak ke-1 artinya bandara ini memiliki *Denied Claim Rate* yang tinggi di antara yang lainnya. Begitu juga dengan American Airlines karena memiliki **proporsi klaim yang ditolak terbanyak ke-2** dan memiliki jumlah klaim terbanyak ke-2 dari total baris dengan 22038 klaim
* Bandara dengan proporsi **klaim yang disetujui terbanyak** (*Approve in Full* + *Settle*) jatuh pada Airline Northwest Airlines dengan 25.08 + 23.64= `48.72%`
* Bandara dengan proporsi **klaim yang disetujui secara penuh (*Approve in Full*)** jatuh pada Airline Southwest Airlines dengan nilai `26.66%`
* Bandara dengan proporsi **klaim yang disetujui secara sebagian (*Settle*)** jatuh pada Northwest Airlines dengan nilai `23.64%`

Selanjutnya kita akan melihat distribusi `Disposition` berdasarkan `Claim Site`

In [ ]:
fig = px.histogram(df, x='Claim Site', facet_row='Disposition', facet_col_wrap=3)
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
fig.show()

Jika kita lihat distribusi `Claim Site` tidak seimbang (Imbalanced) antara *Checkpoint*, *Checked Baggage* dan yang lainnya. Disini kita akan buat kategori yang lainnya ini (*Other*, *Motor Vehicle*, dan *Bus Station*) sebagai kategori baru yaitu *Non-CheckingSite* 

In [ ]:
df.loc[~df['Claim Site'].isin(['Checkpoint', 'Checked Baggage']), 'Claim Site'] = "Non-CheckingSite"

In [ ]:
fig = px.histogram(df, x='Claim Site', color='Disposition')
fig.show()

In [ ]:
df_ct5 = pd.crosstab(df['Claim Site'], df['Disposition'])
df_ct5['Total'] = df_ct5.sum(axis=1)
df_ct5

In [ ]:
px.imshow(df_ct5.iloc[:,0:3], width=600, color_continuous_scale='blues', aspect='auto', height=600)

Dari hasil cross tabulasi di atas dan grafik dapat kita lihat beberapa hal berikut :

* Proporsi yang paling banyak (dari total klaim yaitu) berasal dari kategori *Checked Baggage* yaitu `149242 `baris dari total `189591` baris
* Pada lokasi *Checked Baggage* ternyata hanya `84525/149242` atau `56.60%` yang ditolak artinya cukup banyak klaim yang diterima, padahal mayoritas laporan klaim berasal dari site ini. Site ini juga menjadi site dengan proporsi klaim ditolak paling banyak di antara site yang lain
* Tipe klaim yang proporsi di setujui secara penuh (*Approve in Full*) paling banyak berasal dari lokasi(*Checkpoint*) yaitu `12738/36580` atau `34.82%`
* Tipe klaim yang proporsi di setujui secara partial/sebagian (*Settle*) paling banyak berasal dari lokasi (*Non-CheckingSite*) yaitu `1410/3769` atau `37.41%`

Selanjutnya kita akan melakukan uji statistik apakah proporsi dari setiap kelompok keputusan akhir (final settlement) yaitu `Disposition` memiliki hubungan yang signifikan dengan kelompok site klaim (`Claim Site`)

In [ ]:
df_ct5.drop('Total', axis=1, inplace=True)
df_ct5.columns = ["Approve in Full","Deny","Settle"]
df_ct5.index = ["Checked Baggage", "Checkpoint", "Non-CheckingSite	"]

observed = df_ct5.iloc[0:3,0:3] 
observed

In [ ]:
# uji statistik, apakah proporsi dari setiap kategori kolom `Disposition` memiliki hubungan yang signifikan dengan kategori kolom Claim Site`
# karena kita akan membandingkan dua variabel kategori kita akan gunakan Chi-Squared test
from scipy.stats import chi2_contingency
stats,pvalue,*_=chi2_contingency(observed)
if pvalue <= 0.05:
    print('Tolak Ho')
else:
    print('Gagal menolak Ho')

Ternyata memang ada hubungan yang kuat/signifikan antara proporsi setiap kelompok pada kolm `Disposition` dengan kelompok site klaim (`Claim Site`)

In [ ]:
df['Airport Name'] = df['Airport Name'].str.strip()

In [ ]:
df['claim_id'] = list(range(0,len(df)))
df.to_csv('tsa_claims_clean.csv', index=False)

# Kesimpulan dan Rekomendasi 

Dari analisis yang telah dilakukan, kita bisa membuat kesimpulan berikut tentang keputusan akhir dari klaim yang diajukan ke TSA:
* Dari 189,591 data yang kita miliki,102,392 data berasal dari klaim dengan keputusan akhir nya *Deny*, 46,960 baris dari keputusan akhir nya *Approve in Full* dan 40,293 baris dari keputusan akhir nya *Settle*
* laporan klaim paling banyak terjadi di bulan januari dan kejadian/peristiwa terjadi cukup banyak di bulan desember.
* Klaim masuk paling banyak terjadi setelah 10-11 hari semenjak waktu kejadian. 
* Berdasarkan dana klaim yang diajukan, klaim yang di setujui secara penuh (**Approve in Full**) paling banyak saat dana klaim (`Claim Amount`) yang diajukan memiliki angka uang cukup kecil ($10-$100). Klaim yang ditolak (**Deny**) paling banyak ada di kategori claim amount yang sangat kecil (0-$10) dan di kategori Very Large ($100k-$300B). Sedangkan klaim dengan status (`Disposition`) **Settle** paling banyak berasal dari kategori Medium ($100-$500), Large ($500-$100k) dan Very Large ($100k-$300B)
* Dari semua klaim yang masuk, mayoritas karena penumpang kehilangan benda/properti (*Passenger Property Loss*) 
* Mayoritas klaim berasal dari bandara Los Angeles International Airport
* Maskapai dengan jumlah klaim terbanyak jatuh pada Delta Airlines dengan tingkat penolakan klaim yang tinggi
* Mayoritas laporan klaim berasal dari lokasi *Checked Baggage* yaitu `149242 `baris dari total `189591` baris

Karakteristik klaim yang disetujui (*Approve in Full* dan *Settle*), terutama jika dibandingkan dengan klaim yang ditolak (*Deny*) :
* Biasanya jika dana klaim yang diajukan (`Claim Amount`) berada pada rentang yang cukup kecil $10-$100
* Mayoritas berasal dari jika tipe klaim nya berupa kehilangan barang (*property loss*)\
* Biasanya berasal dari bandara yang relatif kecil jika dibandingkan dengan bandara lainnya di USA. Misalnya Seattle-Tacoma International [Wikipedia](https://en.wikipedia.org/wiki/Seattle%E2%80%93Tacoma_International_Airport) yang memiliki *rate* klaim disetujui tertinggi di antara bandara yang lainya (dari hasil analisis). 
* Waktu pelaporan tidak terlalu jauh dari tanggal kejadian (rentang 7-14 hari)


**Rekomendasi**
1. Tingkatkan keamanan pada di akhir desember, sehingga laporan barang rusak/hilang berkurang.
    * Akan lebih baik membandingkan lagi dari bulan yang lainnya juga
2. Antisipasi laporan klaim yang akan masuk setelah 10-11 hari senjak ada laporan kejadian barang rusak/hilang atau kejadian lainnya.
3. Perlu dibuat sistem *screening* yang lebih baik dan jelas ketika dana klaim pada laporan yang masuk berada pada rentang nilai yang kecil 0-$10. Pada rentang nilai ini, kemungkinan laporan palsu sangat tinggi.
4. Karena klaim yang masuk mayoritas disebabkan oleh kehilangan harta benda penumpang, fokus pada keamanan di lokasi penyimpanan barang seperti bagasi pesawat atau gudang sementara bandara.
5. Perlu dilakukan evaluasi pada bandara John F. Kennedy International dan Delta Airlines mengenai keamanan dan kehandalan penanganan klaim.
    * Akan lebih baik dilakukan analisis lebih mendalam mengenai ini, karena bisa saja bandara lain juga memiliki keamanan yang kurang baik. Terutama bandara dengan kesibukan tinggi.
6. Tingkatkan keamanan pada lokasi *Checked Baggage*

Diharapkan analisis ini bisa membantu untuk menekan biaya kompensasi dan waktu pengecekan klaim, mendeteksi adanya klaim fiktif atau palsu, memberikan evaluasi kepada bandara dalam hal penanganan klaim dan meningkatkan kualitas dengan membantu mengelompokkan klaim ke dalam keputusan akhir (*final settlement*) yang sesuai.